# 특정단어가 언급된 트위터 개수 크롤링하기 - 한 달씩 (ex. 연남동)

참고 : [왕형준님 블로그](https://medium.com/@whj2013123218/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-twitter-%ED%81%AC%EB%A1%A4%EB%A7%81-576f7b098daf)

중복되는 코드를 없앴고 년도별 데이터를 보는 우리 프로젝트에 맞게 코드를 조금 수정했다

In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import  FirefoxBinary
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities
import time
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd
from dateutil.relativedelta import relativedelta

아래 코드를 실행시키기 위해서는

1) Firefox 브라우저가 깔려있어야 합니다.

2) geckodriver.exe가 현재 디렉토리에 있는지 확인해주세요.


In [3]:
def total_twitter_permonth(keyword,startyear,endyear):
    df = pd.DataFrame(data={'Date':[],'Frequency':[]})
    for y in range(startyear,endyear+1):
        startdate = dt.date(year=y,month=1,day=1)
        untildate = dt.date(year=y,month=2,day=1)
        
        if y == 2018 :
            enddate = dt.date(year=y,month=9,day=1)
        else :
            enddate = dt.date(year=y+1,month=1,day=1)

        totalfreq=[]
        while not enddate == startdate:
            url='https://twitter.com/search?&q='+str(keyword)+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typd'
            browser.get(url)

            lastHeight = browser.execute_script("return document.body.scrollHeight")  # 스크롤 내리기 전 위치
            while True:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 스크롤을 맨 아래까지 내림
                time.sleep(2)  # 로딩하는 시간때문에 sleep을 걸어줍니다.

                newHeight = browser.execute_script("return document.body.scrollHeight")  # 현재 스크롤 위치

                if newHeight == lastHeight:  # 더이상 로딩될 트윗이 없다면
                    html = browser.page_source
                    soup = BeautifulSoup(html,'html.parser')
                    tweets = soup.find_all("p", {"class": "TweetTextSize"})
                    wordfreq = len(tweets)  # 페이지에 있는 트윗개수
                    dailyfreq = {'Date':startdate,'Frequency':wordfreq}
                    totalfreq.append(dailyfreq)

                    # 다음 작업을 위해 하루 늘려줌                  
                    startdate = untildate
                    untildate += relativedelta(months=1)
                    # 초기화
                    wordfreq = 0
                    dailyfreq = {}
                    break
                lastHeight = newHeight

        df = pd.concat([df,pd.DataFrame(totalfreq)])
    return df

In [ ]:
binary=FirefoxBinary('C:/Program Files/Mozilla Firefox/firefox.exe')
browser=webdriver.Firefox(executable_path='geckodriver.exe',firefox_binary=binary)